In [1]:
from pathlib import Path
from typing import List, Tuple, Type
from glob import glob
from eis1600.gazetteers.Onomastics import Onomastics
from eis1600.onomastics.re_pattern import ABU_ABI, IBN_IBNA, BN_BNT, DIN_DAULA, DATES, PARENTHESIS, PUNCTUATION, SPACES, QUOTES, UMM, BANU_BANI
from eis1600.preprocessing.methods import get_yml_and_MIU_df

In [2]:
def nasab_analysis(file: Path, og: Type[Onomastics]) -> Tuple[List[str], str, str, str]:
    with open(file, 'r', encoding='utf-8') as miu_file_object:
        yml_header, df = get_yml_and_MIU_df(miu_file_object)

    # if not yml_header.is_bio():
    #     return [], '', ''

    idcs = df[df['TOKENS'].isin(og.end())].index
    idx = idcs[0] if idcs.any() else min(99, len(df))

    text = ' '.join(df['TOKENS'][df['TOKENS'].notna()].iloc[:idx - 1])
    text_w_cutoff = f'{file}\n' + ' '.join(df['TOKENS'][df['TOKENS'].notna()].iloc[:idx])

    text_mnpld = text
    text_mnpld = PARENTHESIS.sub(r'\g<1>', text_mnpld)
    text_mnpld = QUOTES.sub('', text_mnpld)
    text_mnpld = DATES.sub('', text_mnpld)
    text_mnpld = PUNCTUATION.sub('', text_mnpld)
    text_mnpld = SPACES.sub(' ', text_mnpld)
    text_mnpld = ABU_ABI.sub('ابو_', text_mnpld)
    text_mnpld = UMM.sub('ام_', text_mnpld)
    text_mnpld = IBN_IBNA.sub(r'ا\g<1>_', text_mnpld)
    text_mnpld = BN_BNT.sub(r'_\g<1>_', text_mnpld)
    text_mnpld = DIN_DAULA.sub(r'_\g<1>', text_mnpld)
    text_mnpld = BANU_BANI.sub('<بنو_', text_mnpld)
    for elem, repl in og.replacements():
        text_mnpld = text_mnpld.replace(elem, repl)

    tokens = text_mnpld.split()
    unknown = [t for t in tokens if '_' not in t and t not in og.total()]

    return unknown, text, text_mnpld, text_w_cutoff

In [3]:
og = Onomastics.instance()
infiles = glob('../../OpenITI_EIS1600_MIUs/training_data/*.EIS1600')[0:100]

In [4]:
res = []
for infile in infiles:
    res.append(nasab_analysis(infile, og))

In [5]:
UNKNOWN, TEXT, MANIPULATED, text_w_cutoff = zip(*res)
unkown_flat = [elem for sub in UNKNOWN for elem in sub]
unkown_flat

['الخانكي',
 'في',
 'عمائم',
 'الامشاطي',
 'أخو',
 'الذي',
 'قبله',
 'وأمه',
 'الصحراوي',
 'مسن',
 'كان',
 'يسكن',
 'بالعقيبة',
 'بنان',
 'تيم',
 'جليل',
 'القمصي',
 'أخو',
 'الرحمن',
 'وأبوهما',
 'كان',
 'من',
 'صحابة',
 'فجفاه',
 'فقال',
 'في',
 'ذلك',
 'شعرا',
 'قال',
 'الأبار',
 'أصله',
 'من',
 'المدينة',
 'وسكن',
 'دمشق',
 'وداره',
 'بناحية',
 'سوق',
 'القمح',
 'شمالي',
 'دكة',
 'القديمة',
 'وكانت',
 'له',
 'بدمشق',
 'دور',
 'هذه',
 'أحدها',
 'وهو',
 'الفدين',
 'قرية',
 'من',
 'أعمال',
 'دمشق',
 'الموقع',
 'السر',
 'ألف',
 'تزوجها',
 'الاخنائى',
 'النبراوى',
 'له',
 'صحبة',
 'كان',
 'من',
 'أبطال',
 'الأعراب',
 'وفرسانهم',
 'مر',
 'به',
 'النبي',
 'صلى',
 'الله',
 'عليه',
 'وسلم',
 'وهو',
 'يحلب',
 'فقال',
 'قاله',
 'الأعمش',
 'رحل',
 'كان',
 'فاضلا',
 'يستحضر',
 'كثيرا',
 'من',
 'المسائل',
 'الفقهية',
 'وناب',
 'في',
 'ببعض',
 'النواحي',
 'وبالقاهرة',
 'ومرض',
 'مدة',
 'طويلة',
 'بوجع',
 'الظهر',
 'بالإسهال',
 'ولاه',
 'على',
 'غازية',
 'أنا',
 'نا',
 'نا',
 'قال',
 'ولاه',
 'غزو